In [35]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [37]:
from src.inference import get_feature_store

In [38]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-05 04:13:39,778 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 04:13:39,784 INFO: Initializing external client
2025-03-05 04:13:39,785 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 04:13:40,362 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214710
Fetching data from 2025-02-04 09:13:39.778694+00:00 to 2025-03-05 08:13:39.778694+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (8.05s) 


In [39]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-04 10:00:00+00:00,2,0
1,2025-02-04 11:00:00+00:00,2,0
2,2025-02-04 12:00:00+00:00,2,0
3,2025-02-04 13:00:00+00:00,2,0
4,2025-02-04 14:00:00+00:00,2,0
...,...,...,...
175135,2025-03-05 04:00:00+00:00,263,4
175136,2025-03-05 05:00:00+00:00,263,19
175137,2025-03-05 06:00:00+00:00,263,70
175138,2025-03-05 07:00:00+00:00,263,128


In [40]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 175140 entries, 0 to 175895
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         175140 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  175140 non-null  int32                  
 2   rides               175140 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [41]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [42]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 175140 entries, 0 to 175895
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         175140 non-null  datetime64[us]
 1   pickup_location_id  175140 non-null  int32         
 2   rides               175140 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.0 MB


In [43]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [44]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,1,...,1,0,0,1,2,0,1,0,210,2025-02-10 03:00:00
1,3,0,0,0,2,1,0,0,2,0,...,4,1,0,0,0,2,0,0,216,2025-03-03 10:00:00
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,147,2025-02-16 14:00:00
3,3,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,217,2025-02-13 06:00:00
4,0,50,1,0,6,34,32,9,3,2,...,2,6,52,20,2,41,37,4,261,2025-02-04 16:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,5,9,8,1,10,12,14,5,10,22,...,14,8,5,10,6,2,17,4,45,2025-02-08 02:00:00
248,90,22,2,51,32,4,17,22,22,6,...,21,41,22,29,24,34,2,131,158,2025-02-05 12:00:00
249,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,47,2025-03-02 07:00:00
250,16,6,126,95,86,29,34,86,65,156,...,37,116,172,154,15,3,17,134,107,2025-02-20 14:00:00


In [45]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-05 04:13:57,431 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 04:13:57,438 INFO: Initializing external client
2025-03-05 04:13:57,438 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 04:13:57,897 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214710
Fetching data from 2025-02-04 09:13:57.431718+00:00 to 2025-03-05 08:13:57.431718+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.18s) 


In [46]:
current_date

Timestamp('2025-03-05 09:13:57.431718+0000', tz='Etc/UTC')

In [47]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-04 10:00:00
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,3,2025-03-04 10:00:00
2,3,2,3,2,1,7,5,3,2,1,...,3,0,0,1,5,16,11,17,4,2025-03-04 10:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-03-04 10:00:00
4,3,2,3,1,2,4,1,0,4,0,...,0,0,0,0,5,7,14,3,7,2025-03-04 10:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,2,0,0,259,2025-03-04 10:00:00
248,1,1,1,0,1,1,0,0,2,0,...,0,0,0,1,1,2,5,1,260,2025-03-04 10:00:00
249,21,34,25,31,37,28,33,48,54,39,...,0,1,1,1,1,9,12,15,261,2025-03-04 10:00:00
250,102,81,88,81,79,97,95,105,104,53,...,1,4,4,16,87,155,104,140,262,2025-03-04 10:00:00


In [48]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-05 04:14:06,486 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 04:14:06,491 INFO: Initializing external client
2025-03-05 04:14:06,491 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 04:14:06,996 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214710


In [49]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [50]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,6.0
3,6,0.0
4,7,2.0
...,...,...
247,259,0.0
248,260,1.0
249,261,16.0
250,262,117.0


In [51]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([236, 237, 186, 161, 162, 138, 170, 239, 140, 142], dtype=int32)